<h1> Multiagent frameworks in action <h1>
<h3>  Practical guide for implementing stateful, multiagent orchestrations using top-4 frameworks. </h3>
<p> </p>


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  
model = os.getenv("AOAI_CHAT_DEPLOYMENT_NAME", "gpt-4o")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
key = os.getenv("AZURE_OPENAI_API_KEY")


dummy_content = "Metric,FY2024,FY2023\nRevenue,$245.12 billion,$211.92 billion\nRevenue Growth (YoY),15.67%,—\nCost of Revenue,$74.11 billion,$65.71 billion\nGross Profit,$171.01 billion,$146.20 billion\nOperating Expenses,$61.58 billion,$57.53 billion\nResearch & Development,$29.51 billion,$27.20 billion\nSG&A,$32.07 billion,$30.33 billion\nOperating Income,$109.43 billion,$88.68 billion\nPretax Income,$107.79 billion,$89.31 billion\nIncome Tax Expense,$19.65 billion,$16.95 billion\nNet Income,$88.14 billion,$72.36 billion\nEPS (Basic),$11.86,$9.72\nEPS (Diluted),$11.80,$9.68\nFree Cash Flow,$74.07 billion,$59.48 billion\nDividend Per Share,$3.00,$2.72\nGross Margin,69.76%,68.99%\nOperating Margin,44.64%,41.84%\nNet Profit Margin,35.96%,34.15%\nFree Cash Flow Margin,30.22%,28.07%\nEBITDA,$129.43 billion,$102.18 billion\nEBITDA Margin,52.80%,48.22%\nShares Outstanding (Basic),7.431 billion,7.446 billion\nShares Outstanding (Diluted),7.469 billion,7.472 billion\n"
def web_search(query: str) -> str:
    """Perform web search and return the first result."""
    content = [dummy_content]
    return content

query = "Create a detailed financial summary report for Microsoft for FY2024"

### Autogen

Before you proceed, I recommend to create a new python virtual environment and install the required packages in the virtual environment. Run the following command `python -m venv .autogen` to create a new environment and 
Activate the environment by pressing `CTRL+Shitf+P` selecting the Python interpreter.

**Multi-agent interaction patterns supported by Autogen**
- **RoundRobinGroupChat**: A team that runs a group chat with participants taking turns in a round-robin fashion (covered on this page). 
- **SelectorGroupChat**: A team that selects the next speaker using a ChatCompletion model after each message. 
- **MagenticOneGroupChat**: A generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. 
- **Swarm**: A team that uses HandoffMessage to signal transitions between agents. 


In this section we will be using `SelectorGroupChat`

![image.png](https://microsoft.github.io/autogen/stable//_images/selector-group-chat.svg)


**Note:** While AutoGen is a powerful tool for building multi-agent systems and is actively used for rapid prototyping and research, it's generally not considered production-ready in its current state. 
https://devblogs.microsoft.com/semantic-kernel/microsofts-agentic-ai-frameworks-autogen-and-semantic-kernel/

In [30]:
## Install the required packages in the virtual environment

# %pip install -q "autogen-agentchat" "autogen-ext[azure]" "python-dotenv" "aiohttp" "ddgs" "langchain-community" "langchain-core" "playwright"

In [6]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat

from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
import json

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    model="gpt-4o-2024-11-20",
    api_version="2024-08-01-preview",
    azure_endpoint=endpoint
)

planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=az_model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations
    You only plan and delegate tasks - you do not execute them yourself.
    When assigning tasks, use this format:
    1. <agent> : <task>
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

search_agent = AssistantAgent(
    name="WebSearchAgent",
    description="An agent that performs web searches to gather information.",
    tools=[web_search],
    system_message="You are a web search agent. \
    Your only tool is search_tool - use it to find information. \
    You make only one search call at a time. \
    Once you have the results, you never analyze data.",
    model_client=az_model_client
)

data_analyst_agent = AssistantAgent(
    name="DataAnalystAgent",
    description="An agent for performing data analysis.",
    model_client=az_model_client,
    tools=[],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide precise summary.
    Use the data provided do not ask for additional data.
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

selector_prompt = """Select an agent to perform task.
{roles}
Current conversation context:
{history}
Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

# Magentic-One is powerful, generalist multi-agent system, while using Magentic-One as a teams pattern we have to ensure the agents have just-enough capabilities to solve the task at hand. 
team = SelectorGroupChat([search_agent, planning_agent, data_analyst_agent], model_client=az_model_client,  termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False)

## load state from disk
if os.path.exists("team_state.json"):
  with open("team_state.json", "r") as f:
    team_state = json.load(f)
    await team.load_state(team_state)

# Run the agent and stream the messages to the console.
async def main() -> None:
   stream = team.run_stream(task=query)
   await Console(stream)
   await az_model_client.close()
   # Serialize the team state to dictionary
   team_state = await team.save_state()
   # Save the team state to a JSON file
   with open("team_state.json", "w") as f:
      json.dump(team_state, f, indent=4, sort_keys=True, default=str)

# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- TextMessage (user) ----------
Create a detailed financial summary report for Microsoft for FY2024
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_SoNv1OdPPYjZWZOfFq8OLnuT', arguments='{"query":"Microsoft financial summary report FY2024 revenue profit growth metrics"}', name='web_search')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content="['Metric,FY2024,FY2023\\nRevenue,$245.12 billion,$211.92 billion\\nRevenue Growth (YoY),15.67%,—\\nCost of Revenue,$74.11 billion,$65.71 billion\\nGross Profit,$171.01 billion,$146.20 billion\\nOperating Expenses,$61.58 billion,$57.53 billion\\nResearch & Development,$29.51 billion,$27.20 billion\\nSG&A,$32.07 billion,$30.33 billion\\nOperating Income,$109.43 billion,$88.68 billion\\nPretax Income,$107.79 billion,$89.31 billion\\nIncome Tax Expense,$19.65 billion,$16.95 billion\\nNet Income,$88.14 billion,$72.36 billion\\nEPS (Basic),$11.86,$9.72\\nEPS (Diluted)

### LangGraph

Before you proceed, I recommend to create a new python virtual environment and install the required packages in the virtual environment. Run the following command `python -m venv .langraph` to create a new environment and 
Activate the environment by pressing `CTRL+Shitf+P` selecting the Python interpreter.

In [115]:
%pip install -qU "langchain" "langgraph" "ddgs" "langchain-openai" "langgraph-supervisor"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from typing_extensions import TypedDict
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import convert_to_messages, HumanMessage, AIMessage
from langgraph_supervisor import create_supervisor
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore

llm = AzureChatOpenAI(
    azure_deployment=model,
    api_version="2023-05-15",
    temperature=0.3,
    model_name=model,
    azure_endpoint=endpoint,
    api_key =key,
)

###################################### Define state structure ####################
    
checkpointer = InMemorySaver() # Other Supported checkpointers are: SQLLite, Postgres, AzureCosmosDb, Redis
store = InMemoryStore() 

########################################## helper functions #####################

def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)

def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")
########################################## Data gathering agent to fetch search results #########################

data_gatherer_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    prompt=(
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="data_gatherer_agent",
)

########################################## Data analysis agent to perform calculations #########################

def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b

analysis_agent = create_react_agent(
    model=llm,
    tools=[add, multiply, divide],
    prompt=(
        "You are an financial data analysis agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Analyze the data provided by the research agent\n"
        "- Provide a clear, informative, detailed financial summary of the findings\n"
        "- After you're done with your tasks, respond to the supervisor directly"
        "- Respond ONLY with the results of your analysis, do NOT include ANY other text."
    ),
    name="analysis_agent",
)

################################################################ Supervisor agent to manage the workflow #########################
workflow = create_supervisor(
    model=llm,
    agents=[data_gatherer_agent, analysis_agent],
    prompt=(
        "You are a supervisor managing two agents:\n"
        "- a research agent. Assign research-related tasks to this agent\n"
        "- a analysis agent. Assign analysis related tasks to this agent \n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
        "In the end of the task, provide a summary of the findings and end with 'TERMINATE'."
    ),
    
).compile(checkpointer=checkpointer,
    store=store)

inputs =  [
    HumanMessage(
        content=query
    )]
config = {"configurable": {"thread_id": "1"}}
state = {'messages': inputs,'thread_id':10232303}
result = workflow.invoke(input=state,config=config)
for m in result["messages"]:
    m.pretty_print()

Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.


================================ Human Message =================================

Create a detailed financial summary report for Microsoft for FY2024
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_data_gatherer_agent (call_xImXT8yksYCzUiBEZcGznuy9)
 Call ID: call_xImXT8yksYCzUiBEZcGznuy9
  Args:
================================= Tool Message =================================
Name: transfer_to_data_gatherer_agent

Successfully transferred to data_gatherer_agent
================================== Ai Message ==================================
Name: data_gatherer_agent

Microsoft FY2024 Financial Summary Report:

**Key Financial Metrics (FY2024 vs FY2023):**
- **Revenue:** $245.12 billion (up from $211.92 billion, 15.67% YoY growth)
- **Cost of Revenue:** $74.11 billion (up from $65.71 billion)
- **Gross Profit:** $171.01 billion (up from $146.20 billion)
- **Operating Expenses:** $61.58 billion (up from $57.53 bi

In [25]:
list(workflow.get_state_history(config))[0]

StateSnapshot(values={'messages': [HumanMessage(content='Create a detailed financial summary report for Microsoft for FY2024', additional_kwargs={}, response_metadata={}, id='3d8f56e0-2739-42c7-83d0-e2fe16b0fc7a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xImXT8yksYCzUiBEZcGznuy9', 'function': {'arguments': '{}', 'name': 'transfer_to_data_gatherer_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 156, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-Bs2rbrd3EmjzXIie794fK0Qtzg63A', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, name='supervisor', id='run--86a2bfed-04f

In [29]:
store.list_namespaces()

[]

### Semantic Kernel

Before you proceed, I recommend to create a new python virtual environment and install the required packages in the virtual environment. Run the following command `python -m venv .semantickernel` to create a new environment and 
Activate the environment by pressing `CTRL+Shitf+P` selecting the Python interpreter.

In [1]:
%pip install -q "semantic-kernel"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, GroupChatOrchestration, RoundRobinGroupChatManager
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import AuthorRole, ChatHistory, ChatMessageContent
from semantic_kernel.agents import GroupChatManager, BooleanResult, StringResult, MessageResult
from semantic_kernel.contents import ChatMessageContent, ChatHistory
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase

llm = AzureChatCompletion(
    deployment_name=model,
    endpoint=endpoint,
    api_key=key
)

class WebSearchPlugin:
    """Plugin for GitHub Web Search operations."""

    @kernel_function
    def web_search(self, query: str) -> str:
        """Perform web search and return the first result."""
        content = [dummy_content]
        return content

def get_agents():
    return [
        ChatCompletionAgent(name="DataGatherer", 
                            instructions="You are a data gatherer. Search for information related to the task. Do not summarize or analyze data.", 
                            description="An agent for gathering data and performing web searches.",
                            plugins=[WebSearchPlugin()], service=llm), 
        ChatCompletionAgent(name="Analyst", 
                            instructions="You are a financial data analysis agent. INSTRUCTIONS: - Analyze the data provided by the research agent - Provide a clear, informative, detailed financial summary of the findings - \
                                After you're done with your tasks, share the summary - Respond ONLY with the results of your analysis, do NOT include ANY other text.  \
                                Once you're done end with 'TERMINATE'.", 
                            description="An agent for performing data analysis.",
                            service=llm)]

def agent_response_callback(message: ChatMessageContent) -> None:
    print(f"**{message.name}**\n{message.content}")


class CustomGroupChatManager(GroupChatManager):
    topic: str = "Financial Summary Report"
    service: ChatCompletionClientBase
    selection_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to select the next participant to speak. "
        "Here are the names and descriptions of the participants: "
        "{{$participants}}\n"
        "if the analyst agent has completed the task, terminate. Do not ask for additional data. "
        "Please respond with only the name of the participant you would like to select."
    )
    
    def __init__(self, topic: str, service: ChatCompletionClientBase, **kwargs) -> None:
        """Initialize the group chat manager."""
        super().__init__(topic=topic, service=service, **kwargs)
    
    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """Helper to render a prompt with arguments."""
        prompt_template_config = PromptTemplateConfig(template=prompt)
        prompt_template = KernelPromptTemplate(prompt_template_config=prompt_template_config)
        return await prompt_template.render(Kernel(), arguments=arguments)
    
    async def filter_results(self, chat_history: ChatHistory) -> MessageResult:
        # Custom logic to filter or summarize chat results
        summary = "Summary of the discussion."
        return MessageResult(result=ChatMessageContent(role="assistant", content=summary), reason="Custom summary logic.")

    async def select_next_agent(self, chat_history: ChatHistory, participant_descriptions: dict[str, str]) -> StringResult:
        # Randomly select an agent from the participants
        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.selection_prompt,
                    KernelArguments(
                        topic=self.topic,
                        participants="\n".join([f"{k}: {v}" for k, v in participant_descriptions.items()]),
                    ),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Now select the next participant to speak."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )

        participant_name_with_reason = StringResult.model_validate_json(response.content)

        print("*********************")
        print(
            f"Next participant: {participant_name_with_reason.result}\nReason: {participant_name_with_reason.reason}."
        )
        print("*********************")

        if participant_name_with_reason.result in participant_descriptions:
            return participant_name_with_reason

        raise RuntimeError(f"Unknown participant selected: {response.content}.")

    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        # Custom logic to decide if user input is needed
        return BooleanResult(result=False, reason="No user input required.")

    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        # Optionally call the base implementation to check for default termination logic
        base_result = await super().should_terminate(chat_history)
        if base_result.result:
            return base_result
        # Custom logic to determine if the chat should terminate
        should_end = len(chat_history.messages) > 10 or "TERMINATE" in chat_history.messages[-1].content
        return BooleanResult(result=should_end, reason="Custom termination logic.")

async def main():
    agents = get_agents()
    group_chat = GroupChatOrchestration(
        members=agents,
        manager=CustomGroupChatManager(max_rounds=5, topic="Financial Summary Report", service=llm),
        agent_response_callback=agent_response_callback,
    )
    runtime = InProcessRuntime()
    runtime.start()
    result = await group_chat.invoke(
        task=query,
        runtime=runtime,
    )
    value = await result.get()
    print(f"***** Final Result *****\n{value}")
    await runtime.stop_when_idle()

await main()

*********************
Next participant: DataGatherer
Reason: Since financial data needs to be gathered, DataGatherer is selected to collect the relevant data for the report..
*********************
**DataGatherer**

**DataGatherer**

**DataGatherer**
The financial summary details for Microsoft for FY2024 are provided below:

### Key Financial Metrics:

1. **Revenue**: $245.12 billion
   - Year-over-Year (YoY) Revenue Growth: 15.67%

2. **Cost of Revenue**: $74.11 billion
3. **Gross Profit**: $171.01 billion

---

### Operating Expenses:

1. **Total Operating Expenses**: $61.58 billion
   - Research & Development: $29.51 billion
   - Sales, General & Administrative (SG&A): $32.07 billion

---

### Income:

1. **Operating Income**: $109.43 billion
2. **Pretax Income**: $107.79 billion
3. **Income Tax Expense**: $19.65 billion
4. **Net Income**: $88.14 billion
   - Earnings Per Share (Basic): $11.86
   - Earnings Per Share (Diluted): $11.80

---

### Cash Flow:

1. **Free Cash Flow**: $74.

### CrewAI

Before you proceed, I recommend to create a new python virtual environment and install the required packages in the virtual environment. Run the following command `python -m venv .crewai` to create a new environment and Activate the environment by pressing `CTRL+Shitf+P` selecting the Python interpreter.

In [ ]:
%pip install -q crewai-tools crewai langchain_openai

In [32]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
from crewai import Agent, Task, Crew, LLM, Process
import os
from langchain_openai import AzureChatOpenAI
from crewai.tools import tool
from crewai.utilities.paths import db_storage_path
from pathlib import Path


os.environ['AZURE_API_KEY'] = key
os.environ["AZURE_API_BASE"] = endpoint

@tool("web_search")
def web_search(query: str) -> str:
    """Perform web search and return the first result."""
    content = [dummy_content]
    return content

data_gatherer_agent = Agent(
    role='DataGatherer',
    goal=f'Based on the query provided, search and find relevant content ? Query - {query}',
    backstory="You are a data gatherer. Search for information related to the task. Do not summarize or analyze data.",
    verbose=True,
    llm=LLM(model=f'azure/{model}'),
    allow_delegation=False,
    tools=[web_search],
)

analyst_agent = Agent(
    role='DataAnalyst',
    goal='Analyze the data provided by the research agent and provide a clear, informative, detailed financial summary of the findings.',
    backstory="You are a financial data analysis agent. Analyze the data provided by the research agent. Provide a clear, informative, detailed financial summary of the findings.",
    verbose=True,
    llm=LLM(model=f'azure/{model}'),
    allow_delegation=False,
    tools=[],
)

research_task = Task(
    description="""
        Conduct a thorough research about Microsoft's financial performance in FY2024.
        Make sure you find any interesting and relevant information given
        the current year is 2024.
    """,
    expected_output="""
        A list with 10 bullet points of the most relevant information about Microsoft's financial performance in FY2024.
    """,
    agent=data_gatherer_agent
)

reporting_task = Task(
    description="""
        Review the context you got and expand each topic into a full section for a report.
        Make sure the report is detailed and contains any and all relevant information.
    """,
    expected_output="""
        A fully fledge reports with the mains topics, each with a full section of information.
    """,
    agent=analyst_agent,
    markdown=True,  # Enable markdown formatting for the final output
    output_file="report.md"
)

# Store in project directory
current_dir = os.getcwd()
storage_dir = Path(current_dir) / "crewai_storage"
os.environ["CREWAI_STORAGE_DIR"] = str(storage_dir)
storage_path = db_storage_path()
print(f"CrewAI storage location: {storage_path}")
crew = Crew(
    agents=[data_gatherer_agent, analyst_agent],
    tasks=[research_task, reporting_task],
    process=Process.sequential,  # Use sequential process for task execution
    memory=True,  # Enables short-term, long-term, and entity memory
    embedder={
        "provider": "openai",  # Use openai provider for Azure
        "config": {
            "api_key": key,  # Azure OpenAI API key
            "api_base": endpoint,
            "api_type": "azure",
            "api_version": "2023-05-15",
            "model": "text-embedding-3-small",
            "deployment_id": "text-embedding-3-small"  # Azure deployment name
        }
    },
    manager_llm=LLM(model=f'azure/{model}'),
)

output = crew.kickoff()
print(output)

CrewAI storage location: c:\code\sriksml\genai-labs\crewai_storage


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: DataGatherer                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Conduct a thorough research about Microsoft's financial performance in FY2024.                         │
│          Make sure you find any interesting and relevant information given                                      │
│          the current year is 2024.                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: DataGatherer                                                                                            │
│                                                                                                                 │
│  Thought: Thought: To gather relevant financial information about Microsoft's performance in FY2024, I will     │
│  conduct a web search to retrieve the latest and most comprehensive data available.                             │
│                                                                                                                 │
│  Using Tool: web_search                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"Microsoft financial performance FY2024 report\"}"                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  ['Metric,FY2024,FY2023\nRevenue,$245.12 billion,$211.92 billion\nRevenue Growth (YoY),15.67%,—\nCost of        │
│  Revenue,$74.11 billion,$65.71 billion\nGross Profit,$171.01 billion,$146.20 billion\nOperating                 │
│  Expenses,$61.58 billion,$57.53 billion\nResearch & Development,$29.51 billion,$27.20 billion\nSG&A,$32.07      │
│  billion,$30.33 billion\nOperating Income,$109.43 billion,$88.68 billion\nPretax Income,$107.79 billion,$89.31  │
│  billion\nIncome Tax Expense,$19.65 billion,$16.95 billion\nNet Income,$88.14 billion,$72.36 billion\nEPS       │
│  (Basic),$11.86,$9.72\nEPS (Diluted),$11.80,$9.68\nFree Cash Flow,$74.07 billion,$59.48 billion\nDividend Per   │
│  Share,$3.00,$2.72\nGross Margin,69.76%,68.99%\nOperating Margin,44.64%,41.84%\nNet Profit                      │
│  Margin,35.96%,34.15%\nFree Cash Flow Margin,30.22%,28.07%\nEBITDA,$129.43 billion,$102.18 billion\nEBITDA      │
│  Margin,52.80%,48.22%\nShares Outstanding (Basic),7.431 billion,7.446 billion\nShares Outstanding               │
│  (Diluted),7.469 billion,7.472 billion\n']                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

c:\code\sriksml\.crewai\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
c:\code\sriksml\.crewai\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: DataAnalyst                                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Review the context you got and expand each topic into a full section for a report.                     │
│          Make sure the report is detailed and contains any and all relevant information.                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: DataAnalyst                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Financial Performance Report for FY2024                                                                      │
│                                                                                                                 │
│  ## Revenue Overview                                                                                            │
│  - **Total Revenue**: Microsoft achieved a total revenue of **$245.12 billion** in FY2024, up from **$211.92    │
│  billion** in FY2023.                                                                                           │
│  - **Revenue Growth (YoY)**: This represents a **15.67% year-over-year (YoY) growth**, signifying an            │
│  impressive expansion in the company’s top line. The growth was likely fueled by strong demand across its core  │
│  business segments, strategic investments in cloud computing and artificial intelligence, and expected          │
│  increases in customer base and subscription services.                                                          │
│                                                                                                                 │
│  *Revenue growth above 15% showcases the company’s resilience in a competitive landscape and its ability to     │
│  capitalize on market opportunities, reflecting robust product offerings and service innovation.*               │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Cost of Revenue                                                                                             │
│  - **Cost of Revenue**: The cost of generating revenue rose to **$74.11 billion**, compared to **$65.71         │
│  billion** in FY2023.                                                                                           │
│  - The increase of **$8.40 billion**, a **12.78% rise YoY**, suggests that while revenue grew significantly,    │
│  there was also a proportionate rise in costs, potentially driven by:                                           │
│    - Rising input costs for software development, cloud deployment, and infrastructure.                         │
│    - A possible increase in hardware sales or operational scale expansions.                                     │
│                                                                                                                 │
│  *Despite the increase in cost of revenue, Microsoft's ability to maintain growth at a higher pace signifies    │
│  good cost management effectiveness.*                                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Gross Profit                                       

c:\code\sriksml\.crewai\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
c:\code\sriksml\.crewai\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


```markdown
# Financial Performance Report for FY2024

## Revenue Overview
- **Total Revenue**: Microsoft achieved a total revenue of **$245.12 billion** in FY2024, up from **$211.92 billion** in FY2023. 
- **Revenue Growth (YoY)**: This represents a **15.67% year-over-year (YoY) growth**, signifying an impressive expansion in the company’s top line. The growth was likely fueled by strong demand across its core business segments, strategic investments in cloud computing and artificial intelligence, and expected increases in customer base and subscription services.

*Revenue growth above 15% showcases the company’s resilience in a competitive landscape and its ability to capitalize on market opportunities, reflecting robust product offerings and service innovation.*

---

## Cost of Revenue
- **Cost of Revenue**: The cost of generating revenue rose to **$74.11 billion**, compared to **$65.71 billion** in FY2023.
- The increase of **$8.40 billion**, a **12.78% rise YoY**, suggests that w